In [ ]:
import pandas as pd
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, Slider, CustomJS
from bokeh.layouts import column
from bokeh.palettes import Category10

# read data
data = pd.read_csv('gapminder.csv')
data['pop_scaled'] = (data['pop'] / data['pop'].max()) * 50 + 5 #rescaled pop

years = sorted(data['Year'].unique())
initial_year = years[0]

# Get all regions
regions = list(data['Region'].unique())
palette = Category10[10] if len(regions) <= 10 else Category10[10] * ((len(regions) // 10) + 1)

# Create for each region ColumnDataSource
sources = {}
for i, region in enumerate(regions):
    df_region = data[(data['Year']==initial_year) & (data['Region']==region)]
    sources[region] = ColumnDataSource(data=dict(
        x=df_region['Fertility'],
        y=df_region['lifeExp'],
        country=df_region['Country'],
        population=df_region['pop'],
        pop_size=df_region['pop_scaled'],
        year=df_region['Year']
    ))


p = figure(title=f'Life Expectancy vs Fertility ({initial_year})',
           x_axis_label='Fertility (Children per Woman)',
           y_axis_label='Life Expectancy (Years)',
           width=800, height=600)

# scatter
for i, region in enumerate(regions):
    p.scatter(x='x', y='y', size='pop_size', fill_alpha=0.6, line_color='black',
              fill_color=palette[i], legend_label=region, source=sources[region], name=region)

# HoverTool
hover = HoverTool(tooltips=[
    ("Country", "@country"),
    ("Year", "@year"),
    ("Life Expectancy", "@y{0.0}"),
    ("Fertility", "@x{0.00}"),
    ("Population", "@population{0,0}"),
])
p.add_tools(hover)
# HoverTool: Bokeh's hover tool.
    # Function: When the mouse hovers over a certain data point on the chart, additional information (tooltip) is displayed.
    # p.add_tools(hover) : Add this tool to chart p.
# tooltips(label, value): defines the content displayed when hovering; 
    #lable:The displayed text; 
    #value: The bound data field, @column name in ColumnDataSource.

p.legend.title = "Region"
p.legend.location = "top_right"
p.legend.click_policy = "hide"


# slider (start:min, end:max, value:Initial default value)
slider = Slider(start=years[0], end=years[-1], value=initial_year, step=1, title="Year")

# JS callback updates the data of each region
callback = CustomJS(
    args=dict(sources=sources, slider=slider, full_data=data.to_dict('list')), 
    code="""
    const year = slider.value;
    const data = full_data;
    const regions = Object.keys(sources); 
    
    for (let i=0; i<regions.length; i++){
        const region = regions[i];
        const src = sources[region].data;
        const new_x = [], new_y = [], new_country = [], new_population = [], new_pop_size = [], new_year = [];
        for (let j=0; j<data['Year'].length; j++){
            if(data['Year'][j] === year && data['Region'][j] === region){
                new_x.push(data['Fertility'][j]);
                new_y.push(data['lifeExp'][j]);
                new_country.push(data['Country'][j]);
                new_population.push(data['pop'][j]);
                new_pop_size.push(data['pop_scaled'][j]);
                new_year.push(data['Year'][j]);
            }
        }
        src['x'] = new_x;
        src['y'] = new_y;
        src['country'] = new_country;
        src['population'] = new_population;
        src['pop_size'] = new_pop_size;
        src['year'] = new_year;
        sources[region].change.emit();
    }
""")
slider.js_on_change('value', callback)

# full_data=data.to_dict('list') :Convert to a dict, but each key corresponds to a list; (Outer layer)dict+(inner layer)list; e.g. data['Country'][0]  // "China"
# code: Actual JavaScript code, run on the Bokeh front end.
# Object.keys(sources):get the list of region = list(sources.keys()) in Python
# change.emit(): Tell Bokeh that the data source has been updated. Automatically re-scatter points in the current area.
# slider.js_on_change('value', callback): when the value of the slider changes, trigger JS callback.
    # 1.Get the slider year
    # 2.Filter the data corresponding to the year
    # 3.Update the ColumnDataSource in each region
    # 4.The chart automatically refreshes to display the data of the New Year

#Bokeh:Python 生成 HTML → 浏览器执行 JS → JS 回调更新数据 → BokehJS 重绘图形
#Bokeh:Python generates HTML → Browser executes JS → JS callback updates data → BokehJS redraws graphics

#JavaScript:
#let: Variable;
#const: constant (cannot be reassigned);
#JS arryay ~ Python list
    #push: Add an element at the end;
    #index acess: arr[0];
    #if: == Equal values(will automatically convert types);=== Both the value and type are equal.
    #for: 
#JS object ~ Python dict
    #Create an object: const obj = {a: 1, b: 2};
    #Obtain the list of keys: Object.keys(obj);
    #Read object properties: obj["a"]; or obj.a;
        #const regions = Object.keys(sources);
        #sources:{'Asia': ColumnDataSource1,'Europe': ColumnDataSource2,...}
        #regions:["Asia", "Europe", ...]

#BokehJS:
#sources[region].change.emit():Tell Bokeh that the data source has been updated; BokehJS will redraw;
# .change.emit() ~ Python source.data = new_data;

#loop:
#Outer layer: By region (since each region has an independent data source);
#Inner layer: Scan all rows and pick out the data that belongs to the region and the current year;
        
    
# output HTML
output_file("life_expectancy_fertility_interactive.html")
show(column(p, slider))
# output_file("xxx.html"): Set the output target to an HTML file
# column(p, slider): layout function of Bokeh, arranges p (graph) and slider (control) vertically from top to bottom;
# show(...): write content into HTML, and open in a browser.

output_notebook()
